**If you lost points on the last checkpoint you can get them back by responding to TA/IA feedback**  

Update/change the relevant sections where you lost those points, make sure you respond on GitHub Issues to your TA/IA to call their attention to the changes you made here.

Please update your Timeline... no battle plan survives contact with the enemy, so make sure we understand how your plans have changed.

# COGS 108 - UCSD Students Time and Course Satisfaction (Data Checkpoint)

# Names

- Henry Lam
- Alex Truong
- Dianne Natanauan
- Darryl Remulla
- Sam Hormozian

# Research Question

What is the relationship between time spent in courses and UCSD undergraduates’ satisfaction with courses and professors over the time period of Summer Session 2007 to Spring 2023? Does this relationship differ by departments and are there other variables that show stronger correlation with these measures of satisfaction?

## Background and Prior Work


Considering our topic, we wanted to observe how students spend their time for their courses and how they relate to their satisfaction with their course and professor. While discussing how students allocate their time in regards to both academic and other activities, Coffey and his team concluded that “Student GPA is statistically significant and positively related to the probability that a student chooses a given alternative. This result is intuitive and indicates students are concerned about GPA when making time allocation choices.” <a name="cite_ref-1"></a>[<sup>1</sup>](#cite_note-1) For the alternative options Coffey observed other than studying, Coffey surveyed how many weekly hours were devoted by students for Greek activities, unstructured social activities, fitness and recreation, as well as other activities such as staying home and relaxing. Alongside surveying how much time was allocated into these activities by all students, Coffey also observed students whose GPA were above and below the median GPA of the sample they selected. This informs us that how much time a student spent may correlate to their drive for improving their GPA, which may be important in determining whether or not their time allocation is also correlated with their satisfaction of the course and professor. Furthermore, Ferguson and DeFeclice found that students’ academic performance and rating of perceived learning in an online course was increased in a shortened 5-week version of the course compared to the standard full semester course, showing that there is a correlation between time allocated to a course and the students’ satisfaction and performance in that course.
<a name="cite_ref-2"></a>[<sup>2</sup>](#cite_note-2)\
\
To observe UCSD undergraduates' satisfaction with professors and courses, UCSD promoted a system known as CAPES, which stands for Course and Professor Evaluations, where students would provide their opinions and feedback on the course and professor near the final weeks of taking part in the course. Similarly to Coffey's observations, CAPEs surveyed students on how much time they put into the course, whether or not they recommend the course and professor, what their expected grade was, and afterwards when grades were finalized, included the grade received. Although this does not observe every time allocation a particular student decides to spend their time on, it provides us a stepping stone into answering our research question regarding student satisfaction being correlated to their time spent in the respective course with the respective professor.

1. <a name="cite_note-1"></a> [^](#cite_ref-1) https://ageconsearch.umn.edu/record/303903 
2. <a name="cite_note-2"></a> [^](#cite_ref-2)https://www.researchgate.net/publication/44099000_Length_of_Online_Course_and_Student_Satisfaction_Perceived_Learning_and_Academic_Performance

# Hypothesis


We believe as the amount of time spent by UCSD undergraduate students increases, students will be less satisfied by their course and professor.  This outcome is supported by the idea that students do not prefer professors or courses that assign time-consuming work or that time-consuming work would make a class be perceived as tougher.

# Data

## Data overview

For each dataset include the following information
- Dataset #1
  - Dataset Name: UCSD CAPEs Data
  - Link to the dataset: https://www.kaggle.com/datasets/sanbornpnguyen/ucsdcapes?utm_medium=social&utm_campaign=kaggle-dataset-share
  - Number of observations: 63363
  - Number of variables: 11

We are using a dataset of UCSD CAPEs Data from Summer Session 2007 to Spring 2023. The data includes the ratings of 63,363 courses and the respective professors teaching those courses during the duration of that time. The dataset measures total number of students enrolled in the course, the total number of CAPEs given for the course, the percentage of students that would recommend the class, the percentage of students that recommended the professor, the average study hours per week reported by students, average grade expected by students, average grade received by students, and the evaluation URL for that course/professor.

Per our hypothesis, we wanted to focus on the average study hours per week reported by the students and the percentage of students that would recommend the class/professor. Since we assume that there may be a bias in how the course is perceived due to its department or if it is lower or upper division, we want to wrangle our dataset to include two new variable columns: the department a course is in and whether that course is upper or lower division. We also want to remove the evaluation URL column, the letter grade represented in "average grade expected" and "average grade received" by students, and percentage symbol in the related columns  since it did not seem valuable to our analysis.

To create our two new columns, we can split each string in the "Course" column by word, leaving us with the department code in the first split word and the course number in the second split word. Each department code can be  put into a separate column for each course, and each course number can be compared to the number 100 so that the "Upper Division" column would show False for course numbers less than 100 and True for course numbers greater than or equal to it.

## Dataset: UCSD Capes Data from Summer Session 2007 to Spring 2023

## Setup:

In [1]:
## YOUR CODE TO LOAD/CLEAN/TIDY/WRANGLE THE DATA GOES HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION
# Imports 
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
sns.set()
sns.set_context('talk')

import warnings
warnings.filterwarnings('ignore')

import patsy
import statsmodels.api as sm
import scipy.stats as stats
from scipy.stats import ttest_ind, chisquare, normaltest
# Note: the statsmodels import may print out a 'FutureWarning'. Thats fine.

df = pd.read_csv("capes_data.csv")

## Data Cleaning:

In [2]:
def standardize_Upper_Division(st):
    st = st.lower()
    st = st.strip()
    while (st.isnumeric()==False):
        st = st[: len(st)-1]
    st = int(st)
    if (st>99):
        return True
    else:
        return False
def standardize_Percentages(st):
    st = st.lower()
    st = st.strip()
    st = st[: len(st)-1]
    st = float(st)
    return st
def standardize_Grades(st):
    st = str(st)
    if(st == str(df['Average Grade Received'][2])):
        return np.nan
    else:
        return st[len(st)-5:len(st)-1]

In [3]:
#time for some data wrangling: add a Department column and Upper_Division column
df = df.assign(Department = df['Course'].str.split(pat = ' ').str[0])
df = df.assign(Upper_Division = df['Course'].str.split(pat = ' ').str[1])
df['Upper_Division'] = df['Upper_Division'].apply(standardize_Upper_Division)

#remove percentage signs
df['Percentage Recommended Class'] = df['Percentage Recommended Class'].apply(standardize_Percentages)
df['Percentage Recommended Professor'] = df['Percentage Recommended Professor'].apply(standardize_Percentages)

#remove letter grade, leaving only floating points and nan values
df['Average Grade Received'] = df['Average Grade Received'].apply(standardize_Grades)
df['Average Grade Expected'] = df['Average Grade Expected'].apply(standardize_Grades)

#remove 'Evaluation URL' because useless column
df = df[['Instructor', 'Course', 'Quarter', 'Total Enrolled in Course',
       'Total CAPEs Given', 'Percentage Recommended Class',
       'Percentage Recommended Professor', 'Study Hours per Week',
       'Average Grade Expected', 'Average Grade Received',
       'Department', 'Upper_Division']]

#rename some columns to make them easier to work with
df.columns = ['Instructor', 'Course', 'Quarter', 'Total_Enrolled',
       'Total_CAPEs_Given', 'Recommend_Class',
       'Recommended_Professor', 'Weekly_Hours',
       'Avg_Grade_Expected', 'Avg_Grade_Received',
       'Department', 'Upper_Division']

#remove all rows with nan values
df = df.dropna(axis = 'rows')

#change data types for grades expected and received
df = df.astype({'Avg_Grade_Expected': 'float'})
df = df.astype({'Avg_Grade_Received': 'float'})
df

Instructor  \
0      Butler Elizabeth Annette   
1      Butler Elizabeth Annette   
3              Shtienberg Gilad   
4          Braswell Geoffrey E.   
5            Hrvoj Mihic Branka   
...                         ...   
63005     Newsome Elizabeth Ann   
63006            Steinbach Haim   
63007               Wallen Ruth   
63008          Trigilio Michael   
63009          Mangolte Babette   

                                              Course Quarter  Total_Enrolled  \
0        AAS 10 - Intro/African-American Studies (A)    SP23              66   
1                 AAS 170 - Legacies of Research (A)    SP23              20   
3       ANAR 115 - Coastal Geomorphology/Environ (A)    SP23              26   
4      ANAR 155 - Stdy Abrd: Ancient Mesoamerica (A)    SP23              22   
5                     ANBI 111 - Human Evolution (A)    SP23              22   
...                                              ...     ...             ...   
63005  VIS 21A  - Int/ArtAmericas/Africa/Oceania (A)    FA07             230   
63006     VIS 3 - Intr/Art-Making:3-D Practices  (A)    FA07             300   
63007    VIS 60 - Introduction to Photography    (A)    FA07              66   
63008  VIS 70N  - Introduction to Media          (A)    FA07             226   
63009    VIS 84 - History Of Film                (A)    FA07             224   

       Total_CAPEs_Given  Recommend_Class  Recommended_Professor  \
0                     48             93.5                  100.0   
1                      7            100.0                  100.0   
3                      6            100.0                   83.3   
4                      9            100.0                  100.0   
5                      4            100.0                  100.0   
...                  ...              ...                    ...   
63005                160             55.1                   41.1   
63006                220             67.3                   65.7   
63007                 44             90.7                   54.8   
63008                 77             98.7                  100.0   
63009                153             53.9                   32.2   

       Weekly_Hours  Avg_Grade_Expected  Avg_Grade_Received Department  \
0              2.80                3.84                3.67        AAS   
1              2.50                3.86                3.92        AAS   
3              3.83                3.50                3.07       ANAR   
4              5.17                4.00                4.00       ANAR   
5              2.50                4.00                2.95       ANBI   
...             ...                 ...                 ...        ...   
63005          2.63                3.70                3.41        VIS   
63006          4.54                3.65                3.52        VIS   
63007          4.50                3.67                3.36        VIS   
63008          2.71                3.84                3.73        VIS   
63009          3.42                3.23                3.08        VIS   

       Upper_Division  
0               False  
1                True  
3                True  
4                True  
5                True  
...               ...  
63005           False  
63006           False  
63007           False  
63008           False  
63009           False  

[45393 rows x 12 columns]

# Data Cleaning (Description)

- Our data is clean in the sense that there are no nan values.
- To get the data into usable format, made a new column that specified the respective department of each course, made a new column for boolean values that determined Upper_Division, removed excess symbols as well as converted to floats in regards to the data in Recommend_Class and Recommend_Professor and Avg_Grade_Received and Avg_Grade_Expected, and removed nan values.
    - We determined these steps were necessary to get the data into usable format because  comparing relationships will be done between quantitative variables and separating and comparing distributions separated by departments and upper division standing will be easier with these changes.
- Pre-processing steps: we did check data distributions, but no transformations were required since the outliers were dense and might be explained later in the EDA section where we take into account departments and upperdivision standing for courses.

# Ethics & Privacy

Potential Biases: 
- Students might perceive different departments as more difficult or the departments have specific courses that are overall more difficult that might skew the correlation statistics (confounding bias that may result from different student experiences and familarity with specific topics).
- Additionally, students may over-estimate or under-estimate how many hours they needed per week (response bias). 
- Upper-division courses are generally accepted as more difficult than lower division courses, so the difficulty of the course would lower the satisfaction of the course (confounding bias, expectation bias).
- Some courses might inherently require more time for students to learn the material but might not necessarily be more difficult or affect overall satisfaction with the course, which would not be properly represented in our data (measurement bias, since we have no way of measuring if such courses would require more time with the survey data that we have). 
- We don’t believe that there would be issues from our topic area, data, and/or analyses that would be potentially problematic in terms of data privacy and equitable impact.


Handling biases: 
- We could analyze the data separated by department and further separate the data by lower and upper division courses to address the issue of certain departments/upper division classes being perceived as more difficult (matching variables that are more similar to isolate possible confounding variables).
- One way we could account for students potentially over or underestimating their study hours is to observe how their hours impacts their Average Grade Expected as well, so we have another variable to compare their overall satisfaction to. 
    - Additionally, taking a look at the Average Grade Expected will help us account for if a course may require more learning time by design but is not considered too difficult, then a student is likely to expect a higher grade in the course.


# Team Expectations 

1. Communicate times when we are available and let the team know when we are unable to attend a scheduled meeting.
2. Communicate which parts of the project we will work on individually and together, and let the team know when we might need additional help with parts that have been assigned to us.
3. Reach out to team members when they are not completing assigned work by any deadlines we have set, or have not been very responsive when it comes to parts of the project that are highly collaborative.
4. If you work on a section and make a new branch, name it [YourName]-branch with a description of what you worked on.
5. Don’t be afraid to ask for help!

# Project Timeline Proposal

Meeting times will be determined variably through the when2meet app every week.  But tentative project milestones for each week are:
- (completed) Week 6: (2/11 - 2/17): Import, clean, and organize data. (WRANGLE)
- (in progress) Week 7: (2/18 - 2/24): Make graphs that show correlation between data variables/series. (EDA)
- Week 8: (2/25 - 3/2): Make additional comparisons between similar data variables/series to measure correlations.
- Week 9: (3/3 - 3/9): Finalize analysis on generated graphs and descriptive statistics.
- Week 10: (3/10 - 3/16): Turn in Final Project.